# Case. Impact of Agent satisfaction sensibility on under-assignment

Situation:

    More agent availability on week days as weekends, 40% agents work on weekends, 60% not.
    NOT INCLUDED: Shift requirements higher on Friday 3rd Shift (3S), Saturday 1S and 3S, Sunday 3S
    Beta = 0.8

Task:

    Evaluate Shift Coverage over Agent satisfaction penalty for under-assignment
    Evaluate Agent Satisfaction over Agent satisfaction penalty for under-assignment using default function and shift stability function

In [1]:
import abm_scheduling
from abm_scheduling import Schedule as Schedule
from abm_scheduling import Nurse as Nurse

In [2]:
import time
from datetime import datetime

import abm_scheduling.Log
from abm_scheduling.Log import Log as Log

log = Log()

## Randomized. Define situation

In [3]:
p_to_accept_negative_change = .001
num_nurses_per_shift = 5
beta = 0.8

In [4]:
# 25 nurses, only 10 can work on weekends
nurses = []
for n in range(10):
    nurse = Nurse(id_name=n)
    nurse.generate_shift_preferences(degree_of_agent_availability=0.7, works_weekends=True)
    nurses.append(nurse)

In [5]:
for n in range(15):
    nurse = Nurse(id_name=(10+n))
    nurse.generate_shift_preferences(degree_of_agent_availability=0.7, works_weekends=False)
    nurses.append(nurse)

In [6]:
schedule = Schedule(num_nurses_needed=num_nurses_per_shift, is_random=True)
model = abm_scheduling.NSP_AB_Model()
schedule.print_schedule(schedule_name="Intial Situation")

Week's Schedule Intial Situation
+---------+----------+----------+----------+----------+----------+----------+----------+
|         | Mo       | Tu       | We       | Th       | Fr       | Sa       | So       |
+---------+----------+----------+----------+----------+----------+----------+----------+
| shift 1 | need: 6  | need: 3  | need: 7  | need: 4  | need: 6  | need: 6  | need: 6  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 2 | need: 6  | need: 7  | need: 6  | need: 4  | need: 5  | need: 5  | need: 5  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 3 | need: 4  | need: 6  | need: 7  | need: 6  | need: 4  | need: 5  | need: 7  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
+---------+----------+----------+----------+----------+----------+----------+----------+


In [7]:
model.show_hypothetical_max_schedule(schedule=schedule, nurses=nurses, print_detail_schedule = False)

Crude hypothetical shift coverage: 1.0
Shift Coverage Hypothetical Maximum
+---------+------------+------------+------------+------------+------------+-----------+-----------+
|         | Mo         | Tu         | We         | Th         | Fr         | Sa        | So        |
+---------+------------+------------+------------+------------+------------+-----------+-----------+
| shift 1 | need: 6    | need: 3    | need: 7    | need: 4    | need: 6    | need: 6   | need: 6   |
|         | nurses: 21 | nurses: 13 | nurses: 19 | nurses: 15 | nurses: 16 | nurses: 6 | nurses: 7 |
|         | (3.5)      | (4.33)     | (2.71)     | (3.75)     | (2.67)     | (1)       | (1.17)    |
| shift 2 | need: 6    | need: 7    | need: 6    | need: 4    | need: 5    | need: 5   | need: 5   |
|         | nurses: 18 | nurses: 18 | nurses: 17 | nurses: 17 | nurses: 20 | nurses: 8 | nurses: 9 |
|         | (3)        | (2.57)     | (2.83)     | (4.25)     | (4)        | (1.6)     | (1.8)     |
| shift 3 | need

In [8]:
model.get_total_agent_satisfaction(nurses)

-25000.0

In [9]:
# Default utility function (no agent satisfaction)
start_time = datetime.today()
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.8, 
                         p_to_accept_negative_change=.001, utility_function_parameters = None)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

Week's Schedule Best Schedule. Beta: (0.8),p: (0.001)
+---------+--------------+-------------+--------------+-------------+--------------+----------+----------+
|         | Mo           | Tu          | We           | Th          | Fr           | Sa       | So       |
+---------+--------------+-------------+--------------+-------------+--------------+----------+----------+
| shift 1 | need: 6      | need: 3     | need: 7      | need: 4     | need: 6      | need: 6  | need: 6  |
|         | nurses:      | nurses:     | nurses:      | nurses:     | nurses:      | nurses:  | nurses:  |
|         | 23,4,15,1,   | 2,1,19      | 16,10,4,22,  | 9,22,13,12  | 18,12,20,10, | 7,9,4    | 9,7      |
|         | 7,17         |             | 20,12,1      |             | 6,23         |          |          |
| shift 2 | need: 6      | need: 7     | need: 6      | need: 4     | need: 5      | need: 5  | need: 5  |
|         | nurses:      | nurses:     | nurses:      | nurses:     | nurses:      | nurse

In [10]:
model.print_nurse_productivity(results.nurses)

Nurse productivity -  
Nr:    0, 	assigned:5,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.83,	satisf: 533.67,	underbooked
Nr:    1, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:    2, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:    3, 	assigned:4,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.67,	satisf: 281.67,	underbooked
Nr:    4, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:    5, 	assigned:4,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.67,	satisf: 281.67,	underbooked
Nr:    6, 	assigned:7,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.17,	satisf: -166.67,	overbooked
Nr:    7, 	assigned:4,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.67,	satisf: 281.67,	underbooked
Nr:    8, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:    9, 	assigned:5,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.83,	satisf: 533.67,	underbooked
Nr:   10, 	assigned:2,	min:6,	max: 6,	deg.availab:0.7

In [11]:
# Agent satisfaction utility function
utility_function_parameters = abm_scheduling.Utility_Function_Parameters()
utility_function_parameters.utility_function = 'agent_satisfaction'
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.8, 
                         p_to_accept_negative_change=.001, utility_function_parameters = utility_function_parameters)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

Week's Schedule Best Schedule. Beta: (0.8),p: (0.001)
+---------+-------------+--------------+-------------+------------+-------------+----------+----------+
|         | Mo          | Tu           | We          | Th         | Fr          | Sa       | So       |
+---------+-------------+--------------+-------------+------------+-------------+----------+----------+
| shift 1 | need: 6     | need: 3      | need: 7     | need: 4    | need: 6     | need: 6  | need: 6  |
|         | nurses:     | nurses:      | nurses:     | nurses:    | nurses:     | nurses:  | nurses:  |
|         | 4,11,17,12, | 1,21,19      | 1,8,6,20,   | 24,20,9,4  | 23,10,14,0, | 9,4      | 8        |
|         | 23,16       |              | 12,22,13    |            | 2,6         |          |          |
| shift 2 | need: 6     | need: 7      | need: 6     | need: 4    | need: 5     | need: 5  | need: 5  |
|         | nurses:     | nurses:      | nurses:     | nurses:    | nurses:     | nurses:  | nurses:  |
|         

In [12]:
model.print_nurse_productivity(results.nurses)

Nurse productivity -  
Nr:    0, 	assigned:5,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.83,	satisf: 533.67,	underbooked
Nr:    1, 	assigned:5,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.83,	satisf: 533.67,	underbooked
Nr:    2, 	assigned:5,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.83,	satisf: 533.67,	underbooked
Nr:    3, 	assigned:5,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.83,	satisf: 533.67,	underbooked
Nr:    4, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:    5, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:    6, 	assigned:5,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.83,	satisf: 533.67,	underbooked
Nr:    7, 	assigned:4,	min:6,	max: 6,	deg.availab:0.70,	prod: 0.67,	satisf: 281.67,	underbooked
Nr:    8, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:    9, 	assigned:6,	min:6,	max: 6,	deg.availab:0.70,	prod: 1.00,	satisf: 768.60,	ok
Nr:   10, 	assigned:2,	min:6,	max: 6,	deg.availab:0.7

## Targeted search. Define situation

In [ ]:
#%% Initializations
p_to_accept_negative_change = .001

# Situation definition
matrix_nurses_needed = [5,6,3, 5,6,3, 5,6,3, 5,6,3, 5,6,5, 6,6,5, 6,6,3]
#type 1 only first shift work week days
matrix_nurse_availability_type1 = ['x','','', 'x','','',  'x','','', 'x','','', 'x','','', 'x','','', '','','x']
#type 2 1s+2s work week days
matrix_nurse_availability_type2 = ['x','x','', 'x','x','',  'x','x','', 'x','x','', 'x','x','', '','','', '','','']
#type 3 1s+2s+3s work week days
matrix_nurse_availability_type3 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','', 'x','x','', 'x','x','']
#type 4 "springers"
matrix_nurse_availability_type4 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x']
maximum_shifts_type4=3
#type 5 work only weekends
matrix_nurse_availability_type5 = ['','','', '','','', '','','', '','','', '','','x', 'x','x','x', 'x','x','x']


# Create Schedule
#schedule_random = Schedule(matrix_nurses_needed=matrix_nurses_needed, is_random=False)
schedule = Schedule(matrix_nurses_needed=matrix_nurses_needed)

# Create model and nurses
model = abm_scheduling.NSP_AB_Model()
#nurses = model.generate_nurses(10, 0.5, True)
list_nurse_schedules = []
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)

nurses = model.generate_nurses_from_nurse_schedules(list_nurse_schedules)
schedule.print_schedule(schedule_name="Intial Situation")

In [ ]:
nurses[10].minimum_shifts = 2
nurses[10].maximum_shifts = 4
nurses[10].print_shift_preferences()
nurses[11].minimum_shifts = 2
nurses[11].maximum_shifts = 4
nurses[11].print_shift_preferences()

In [ ]:
model.get_total_agent_satisfaction(nurses)

In [ ]:
model.print_nurse_productivity(nurses)

In [ ]:
model.show_hypothetical_max_schedule(schedule=schedule, nurses=nurses)

In [ ]:
# Default utility function (no agent satisfaction)
start_time = datetime.today()
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = None)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)

In [ ]:
# Agent satisfaction utility function
utility_function_parameters = abm_scheduling.Utility_Function_Parameters()
utility_function_parameters.utility_function = 'agent_satisfaction'
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = utility_function_parameters)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)